In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
import sys
import time
import pandas as pd
import xarray as xr
import pickle

start = time.time()

cwd = os.getcwd()
dirLib = cwd + r'/../library'
if dirLib not in sys.path:
    sys.path.append(dirLib)

import fit_polynomial_utils as fit
import data_organize as dorg

# My simulated

In [3]:
# Only need to change this file location if using simulated data generated from "generate_sim_data.py"
load_sim_pkl_dir = r'C:\Users\Grant\OneDrive - UCB-O365\ARSENL\Experiments\Deadtime_Experiments\Data\Simulated\Select'
fname_sim_pkl_ref = r'C:\Users\Grant\OneDrive - UCB-O365\ARSENL\Experiments\Deadtime_Experiments\Data\Simulated\sim_amp1.0E+06_nshot1.0E+06.pkl'
infile_ref = open(fname_sim_pkl_ref, 'rb')
df = pickle.load(infile_ref)
infile_ref.close()

flight_time_ref = xr.DataArray(df.flight_time.data)
true_flight_time_ref = xr.DataArray(df.true_flight_time.data)
n_shots_ref = df.n_shots.data
t_det_lst_ref = df.t_det_lst.data
t_phot_lst_ref = df.t_phot_lst.data
target_amplitude_ref = df.target_amplitude.data
target_time_ref = df.target_time.data
laser_pulse_width_ref = df.laser_pulse_width.data
window_bnd_ref = df.window_bnd.data
background_ref = df.background.data

# netcdf converted from .ARSENL

In [7]:
### PATH VARIABLES ###
load_dir = r'C:\Users\Grant\OneDrive - UCB-O365\ARSENL\Experiments\SPCM\SPCM_Data_2023.01.31\netcdf'  # Where the data is loaded from
fname = r'\OD50_Dev_0_-_2023-01-31_16.43.48_OD5.0.ARSENL.nc'  # The dataset that will serve as the high-fidelity reference when evaluating

In [9]:
ds = xr.open_dataset(load_dir + fname)

In [10]:
ds

<xarray.Dataset>
Dimensions:              (time_tag_index: 17729, sync_index: 1608890)
Coordinates:
  * sync_index           (sync_index) uint64 8003617 8003618 ... 9612505 9612506
Dimensions without coordinates: time_tag_index
Data variables:
    time_tag             (time_tag_index) int64 ...
    time_tag_diff        (time_tag_index) int64 ...
    time_tag_sync_index  (time_tag_index) uint64 ...
    sync_ux_time         (sync_index) datetime64[ns] ...
    sync_ux_time_us      (sync_index) timedelta64[ns] ...
    sync_diff_time       (sync_index) int64 ...
    sync_delta           (sync_index) int64 ...

In [15]:
tt = ds.time_tag
tt_sync_idx = ds.time_tag_sync_index
sync_idx = ds.sync_index

In [16]:
tt

<xarray.DataArray 'time_tag' (time_tag_index: 17729)>
array([1397877, 1786135,    1174, ..., 2370286,  236626,    1180], dtype=int64)
Dimensions without coordinates: time_tag_index
Attributes:
    description:       Photon relative time tag, with 25 ps per count
    units:             clock counts
    clock units:       ps
    device:            dev
    clock conversion:  25.0

In [17]:
tt_sync_idx

<xarray.DataArray 'time_tag_sync_index' (time_tag_index: 17729)>
array([8003747, 8003916, 8004007, ..., 9612224, 9612289, 9612357], dtype=uint64)
Dimensions without coordinates: time_tag_index
Attributes:
    description:  Sync counter value corresponding to the time tag
    units:        none
    device:       dev

In [18]:
sync_idx

<xarray.DataArray 'sync_index' (sync_index: 1608890)>
array([8003617, 8003618, 8003619, ..., 9612504, 9612505, 9612506], dtype=uint64)
Coordinates:
  * sync_index  (sync_index) uint64 8003617 8003618 8003619 ... 9612505 9612506
Attributes:
    description:  Sync counter
    units:        none
    device:       dev